In [1]:
from dotenv import load_dotenv
import os
import re
import pickle
import threading
import itertools
import time
import json
import uuid
from tqdm import tqdm 
import torch 
from collections import Counter

from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.stores import InMemoryByteStore
from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.output_parsers.openai_functions import JsonKeyOutputFunctionsParser
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain_upstage import ChatUpstage, UpstageEmbeddings, UpstageLayoutAnalysisLoader, UpstageGroundednessCheck
from langchain_text_splitters import Language,RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS, Chroma

from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever
from langchain_core.runnables import RunnablePassthrough
from langchain.schema import Document
from datasets import load_dataset
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain import hub
from langchain.tools.retriever import create_retriever_tool
from transformers import AutoTokenizer, AutoModelForSequenceClassification

from utils import format_docs, format_arc_doc

/home/sml09181/anaconda3/envs/jin/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from langchain_engine.langchain_engine import *

In [6]:
from utils import * 
import warnings
from argparse import ArgumentParser
from datasets import load_dataset

warnings.filterwarnings('ignore') 

# Load configs
load_env(".env")
config = load_yaml("config.yaml") 
data_root = config['data_root']
chunk_size = config['chunk_size'] 
chunk_overlap = config['chunk_overlap']
ewha_faiss_path = config['ewha_faiss_path']
ewha_bm25_path = config['ewha_bm25_path'] 
arc_faiss_path = config['arc_faiss_path']
arc_bm25_path = config['arc_bm25_path']
summ_chroma_path = config['summ_chroma_path']
summ_faiss_path = config['summ_faiss_path']
pc_chroma_path = config['pc_chroma_path']
pc_faiss_path = config['pc_faiss_path']
raptor_faiss_path = config['raptor_faiss_path']
top_k = config['top_k']

In [7]:
ewha_ret1 = "faiss"
ewha_ret2 = "pc_chroma"

questions, answers = read_data(data_root, filename="test_samples.csv") 

In [8]:
# returns retriever FAISS 
save_dir="./db/ewha/ewha_faiss_fix"
top_k=4
embeddings = get_embedding()
print("[INFO] Get retriever FAISS ...")

vectorstore1 = FAISS.load_local(save_dir, embeddings, allow_dangerous_deserialization=True) 
print(f"[INFO] Load DB from {save_dir}...") 

retriever1 = vectorstore1.as_retriever(search_kwargs={"k": top_k}) # default = 4



[INFO] Loading embeddings...
[INFO] Get retriever FAISS ...
[INFO] Load DB from ./db/ewha/ewha_faiss_fix...


In [4]:
# allowed_search_types  get_relevant_documents(
for question in questions:
    docs = vectorstore1.similarity_search_with_score(question, k=3)
    print(docs)

[(Document(metadata={'page': 12}, page_content='또는 제2항의 학점 내에서 총장이 정하는 소정의 학점을 영\n어강의로 이수하여야 한다. (신설 2013.11.20)\n⑥ 스크랜튼학부 자유전공 입학생의 졸업에 필요한 학점은 제1전공의 졸업에 필요한 학점\n으로 한다. (신설 2009.2.23.) 제48조의2(영어 및 정보인증) ① 영어 및 정보 등에 관하여 일정한 기준의 능력이나 자격을 취득한 경우 이를 각 3학점으로 인정하고 인증서를 교부할 수 있다. (개정 2015.9.18.) ② 제1항의 시행에 관한 사항은 총장이 따로 정한다. [본조신설 2000.6.20.] [제목개정 2015.'), 0.8335583), (Document(metadata={'page': 8}, page_content='. (개정\n2001.9.24) 제35조의3(학점의 인정) ① 다음 각 호의 경우에 취득한 학점은 총장의 승인을 얻어 졸업에\n필요한 학점의 2분의 1의 범위 안에서 이를 본교에서 취득한 것으로 본다. (개정\n2012.12.31.) 1. 재학 중 국내외의 다른 학교에서 학점을 취득한 경우\n2. 입학 전 국내외의 고등학교와 「고등교육법」 제2조 각 호의 학교에서 대학교육과정에 상\n당하는 교과목을 이수한 경우\n② 학점의 인정범위 등 제1항의 시행에 필요한 사항은 총장이 따로 정한다. (개정\n1998.6.23) ③ 삭제 (1998.6.23.'), 0.9629625), (Document(metadata={'page': 8}, page_content='이화여자대학교 학칙 제35조의2(학점취득의 특례) ① 교양과목과 전공기초과목 중 특정교과목은 이를 수강하지 아\n니하여도 학점취득특별시험에 의하여 학기당 최대취득학점 외에 학점을 취득할 수 있으며, 그 학점수는 12학점을 초과할 수 없다. (개정 1996.2.15., 2017.2.8.)\n② 학점취득특별시험의 방법, 종류에 관한 세부 사항은 총장이 따로 정한다. (개정\n2001.9.24

In [ ]:
save_dir="./db/pc_chroma"
top_k=4
chunk_size=1000
chunk_overlap=100
splits = load_ewha(data_root, chunk_size, chunk_overlap) 
"""Parent Document Retreiver using Chroma"""
embeddings = get_embedding() 
#docstore_path = os.path.join(save_dir, "docstore_pc.pkl")
os.makedirs(save_dir, exist_ok=True) 

# The vectorstore to use to index the child chunks
vectorstore = get_chroma_vs(save_dir, embeddings, "parent-child")

# Layer to store parent document
store = InMemoryByteStore()
id_key = "doc_id"
retriever = MultiVectorRetriever(
        vectorstore=vectorstore,
        byte_store=store,
        id_key=id_key,
        search_kwargs={"k": top_k},
    )

doc_ids = [str(uuid.uuid4()) for _ in splits]

# splitter to make child chunk
child_text_splitter = RecursiveCharacterTextSplitter(
            chunk_overlap=chunk_overlap,
            chunk_size=chunk_size)

sub_docs, data = get_child(splits, doc_ids, child_text_splitter, id_key)

retriever.vectorstore.add_documents(sub_docs)
retriever.docstore.mset(list(zip(doc_ids, splits)))

[INFO] # of splits: 26
[INFO] Loading embeddings...


NameError: name 'id_key' is not defined

In [ ]:
retriever.search_type = SearchType.similarity
retriever.search_type = SearchType.similarity_score_threshold

In [ ]:
retriever.vectorstore.similarity_search("justice breyer")[0]
from langchain.retrievers.multi_vector import SearchType

retriever.search_type = SearchType.mmr

len(retriever.invoke("justice breyer")[0].page_content)

collection = client.create_collection(
        name="collection_name",
        metadata={"hnsw:space": "cosine"} # l2 is the default
    )
res2 = db.similarity_search_with_score(query2, k=3)
db = Chroma.from_documents(texts, embeddings)

docs_score = db.similarity_search_with_score(query=query, distance_metric="cos", k = 6)

In [ ]:
# Load and Split documents
splits = [];arc_data = []
ret_dict = {
    "faiss": [split_docs, get_faiss, ewha_faiss_path],
    "bm25": [split_docs, get_bm25, ewha_bm25_path],
    "pc_faiss": [load_ewha, get_pc_faiss, pc_faiss_path],
    "pc_chroma": [load_ewha, get_pc_chroma, pc_chroma_path],
    "summ_faiss": [load_ewha, get_summ_faiss, summ_faiss_path],
    "summ_chroma": [load_ewha, get_summ_chroma, summ_chroma_path],
    "rap_faiss": [split_docs, get_faiss, raptor_faiss_path],
}

# Make embeddings, db, and rertriever 
splits = ret_dict.get(ewha_ret1)[0](data_root, chunk_size, chunk_overlap) 
ewha_retriever1  = ret_dict.get(ewha_ret1)[1](splits, save_dir=ret_dict.get(ewha_ret1)[2], top_k=top_k, chunk_size=chunk_size, chunk_overlap=chunk_overlap)

if ewha_ret2 is not None:
    splits = ret_dict.get(ewha_ret2)[0](data_root, chunk_size, chunk_overlap) 
    ewha_retriever2  = ret_dict.get(ewha_ret2)[1](splits, save_dir=ret_dict.get(ewha_ret2)[2], top_k=top_k, chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    ewha_retriever_ensemble = get_ensemble_retriever(ewha_retriever1, ewha_retriever2, [0.5, 0.5])